In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

In [2]:
batch_size = 128
epoch = 20
# tensorboard_writer = SummaryWriter()
epsilon=0.01
T_0 =3
T_mult=2
eta_min = 1e-6

In [3]:
class CustomDataset(Dataset):

    def __init__(self, root_dir):
        self.dataset = np.load(root_dir)
        print('number of data points', self.dataset.shape[0])

    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        x = self.dataset[idx, :,:,0]
        y = self.dataset[idx, :,:,1]
        return x,y

In [4]:
train_set = CustomDataset('matrix_2_dataset/train_set.npy')
val_set = CustomDataset('matrix_2_dataset/val_set.npy')
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=batch_size)

number of data points 1000000
number of data points 10000


In [9]:
A=3
B=4
C=1
D=2
matrix_norm = A*D-B*C

train_accu = 0
i = 1
for data, target in val_loader:
    target = target - torch.tensor([[[1,-2],[-0.5,1.5]]])
    target = target/epsilon
    output = np.zeros((batch_size, 2,2), dtype=np.float64)
    for i in range(5):
        a = data[i,0,0]*epsilon
        b = data[i,0,1]*epsilon
        c = data[i,1,0]*epsilon
        d = data[i,1,1]*epsilon
        output[i,0,0] = d/matrix_norm - D*(D*a+A*d-B*c-C*b)/(matrix_norm**2)
        output[i,0,0] = output[i,0,0]/epsilon
        test = (D+d)/(matrix_norm+D*a+A*d+a*d-B*c-C*b-b*c)
        test = (test-1)/epsilon
        print('test', test.numpy())
        print('linear', output[i,0,0])
        print('target', target.view(-1,4)[i,0].numpy(),'\n')
    break

test 0.34058353734560676
linear 0.3399165530296715
target 0.34058353734554014 

test 1.1474247167015283
linear 1.1359026744430605
target 1.1474247167015061 

test -0.5033391239472662
linear -0.5050797018541915
target -0.503339123947244 

test -0.9053894516602101
linear -0.9133210509990004
target -0.9053894516602101 

test -0.6063721622104667
linear -0.610209164663111
target -0.6063721622104445 

